Template notebook for one type of model.

In [23]:
import pandas as pd
from xgboost import XGBClassifier,XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from xgboost import XGBRegressor, XGBClassifier
from statistics import mean
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
# Other imports

In [24]:
# Load data
data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")
data_test = pd.read_csv("test_data_enriched.csv",index_col="Claim Identifier")

C:\Users\Casa-PC\AppData\Local\Temp\ipykernel_17792\1043285265.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")


In [25]:
le = LabelEncoder()

In [26]:
#Label inconding our target variable 
data["Claim Injury Type"] = le.fit_transform(data["Claim Injury Type"])

In [ ]:
def impute_missing_values(X_train,X_val, target_column, algorithm,validation = True):
    # Separating the missing values from the non missing values
    available_data = X_train[X_train[target_column].notna()]
    missing_X_train = X_train[X_train[target_column].isna()]
    missing_X_val = X_val[X_val[target_column].isna()]
    missing_X_val = missing_X_val.drop(columns=[target_column])

    # Diagnóstico inicial
    """print(f"\nImputando valores para coluna: {target_column}")
    print(f"available data: {len(available_data)}")
    print(f"missing data: {len(missing_X_train)}")"""

    # Verificar se há dados suficientes para imputação
    if len(available_data) == 0 or len(missing_X_train) == 0:
        # print(f"nao deu input na coluna {target_column}")
        return X_train, X_val

    # Separating the target column from the rest
    X_available = available_data.drop(columns=[target_column])
    y_available = available_data[target_column]

    # Garantir consistência entre colunas
    X_available = X_available.select_dtypes(include=["number"])
    missing_X_train = missing_X_train.select_dtypes(include=["number"])

    common_columns = X_available.columns.intersection(missing_X_train.columns).intersection(missing_X_val.columns)
    X_available = X_available[common_columns]
    missing_X_train = missing_X_train[common_columns]

    # Verificar se ainda há colunas suficientes após alinhamento
    if X_available.shape[1] == 0:
        # print(f"Sem colunas disponíveis para imputar na coluna {target_column}")
        return X_train, X_val

    # Training the model with the available data
    model = algorithm
    model.fit(X_available, y_available)

    # Prediting the missing values
    predicted_train = model.predict(missing_X_train)
    if validation:
        predicted_val = model.predict(missing_X_val)

    #print(len(predicted_train))

    # Criar uma cópia explícita para evitar problemas com views
    X_train = X_train.copy()
    X_train.loc[X_train[target_column].isna(), target_column] = predicted_train

    if validation:
        X_val = X_val.copy()
        X_val.loc[X_val[target_column].isna(), target_column] = predicted_val

    return X_train, X_val

In [28]:
def check_missing_values(data, step_name):
    print(f"\n{step_name}: Valores ausentes restantes:")
    print(data.isnull().sum()[data.isnull().sum() > 0])

In [29]:
import matplotlib.pyplot as plt
import seaborn as sns

def handle_outliers(data, column):
    """
    Handle outliers in a numerical column by replacing values outside the interquartile range (IQR) with the respective bounds.
    Additionally, generates boxplots before and after the outlier treatment.

    Parameters:
        data (pd.DataFrame): The dataset containing the column.
        column (str): The column name to process.

    Returns:
        pd.DataFrame: DataFrame with treated outliers.
    """
    # Verifica se a coluna possui valores válidos
    if data[column].notnull().sum() > 0:  # Apenas processa colunas com dados válidos
        """# Criar boxplot antes do tratamento
        plt.figure(figsize=(10, 6))
        sns.boxplot(data[column])
        plt.title(f"Boxplot Antes do Tratamento de Outliers - {column}")
        plt.show()"""

        # Calcular limites do IQR
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Substituir valores fora dos limites pelos próprios limites
        data[column] = np.where(data[column] < lower_bound, np.nan, data[column])
        data[column] = np.where(data[column] > upper_bound, np.nan, data[column])

        """# Criar boxplot após o tratamento
        plt.figure(figsize=(10, 6))
        sns.boxplot(data[column])
        plt.title(f"Boxplot Após o Tratamento de Outliers - {column}")
        plt.show()"""

    return data

In [30]:
def scale_numerical(column, X_train, X_val, scaler):
    # Certifique-se de que a coluna é numérica
    if not pd.api.types.is_numeric_dtype(X_train[column]):
        print(f"A coluna '{column}' não é numérica e será ignorada.")
        return

    # Escala os dados e substitui os valores na coluna
    try:
        X_train[column] = scaler.fit_transform(X_train[[column]])
        X_val[column] = scaler.transform(X_val[[column]])
    except ValueError as e:
        print(f"Erro ao escalonar a coluna '{column}': {e}")

In [31]:
def  claim_carrier_categories(X_train, X_val):
    # Define a function to categorize each carrier based on its claim count
    count = X_train['Carrier Name'].value_counts()
    def categorize_claims(count):
        if count >= 40000:
            return 2
        elif 4000 <= count < 40000:
            return 1
        else:
            return 0

    # Apply the categorization to create a mapping dictionary
    carrier_category_map = count.apply(categorize_claims)

    # Map the `Carrier Name` to the new `Carrier Claim Category`
    X_train['Carrier Claim Category'] = X_train['Carrier Name'].map(carrier_category_map)
    X_val['Carrier Claim Category'] = X_val['Carrier Name'].map(carrier_category_map)


In [32]:
# Categorical encoder function
def categorical_prop_encode(X_train, X_val, feature):
    proportion = X_train[feature].value_counts(normalize = True)  # Get the porportion of each category
    X_train[feature] = X_train[feature].map(proportion)  # Map the porportions in the column
    X_val[feature] = X_val[feature].map(proportion) # Do the same for the valid dataset
    X_val[feature] = X_val[feature].fillna(0)  # Handle categories in X_val not seen in X_train with 0


In [33]:
def Rfe(algorithm,X,y,num_features,cat_features,num_imputing_algorithm= XGBRegressor() , cat_imputing_algorithm = XGBClassifier(), scaling_outlier = False, scaler = MinMaxScaler()):

    X_train, X_val,y_train, y_val = train_test_split(X,y,
                                                train_size = 0.8, 
                                                shuffle = True, 
                                                stratify = y)

    #Filling num missing values
    for column in num_features:
        X_train, X_val = impute_missing_values(X_train,X_val, column, num_imputing_algorithm)

    X_train, X_val = impute_missing_values(X_train,X_val, "Alternative Dispute Resolution", cat_imputing_algorithm)

    # Removing inconsistencies on the train
    inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
    X_train = X_train.loc[~X_train.index.isin(inconsistent)]
    y_train = y_train.loc[~y_train.index.isin(inconsistent)]

    # Performing scaling and outlier treatment dependent on the boolean
    if scaling_outlier:
        for column in num_features:
            handle_outliers(X_train, column)
            X_train, X_val = impute_missing_values(X_train,X_val, column, num_imputing_algorithm,validation= False)

        for column in num_features:
            scale_numerical(column, X_train, X_val, scaler)

    # Creating an ordinal variable
    claim_carrier_categories(X_train, X_val)

    X_val, X_train = impute_missing_values(X_val,X_train, "Carrier Claim Category", cat_imputing_algorithm,validation= False)

    # Categorical Prop Encoding
    for cat_feature in cat_features:
        categorical_prop_encode(X_train, X_val, cat_feature)
        if scaling_outlier:
            scale_numerical("Carrier Claim Category", X_train, X_val, scaler)

    #Generating the variables where we will store our results
    nof_list = np.arange(1, len(X_train.columns) + 1)            
    high_score = 0
    opt_n_features = 0
    train_score_list = []
    val_score_list = []

    #Variable where we will store the optimum amount of features
    best_rfe = None

    model = algorithm

    for n in nof_list:
        rfe = RFE(estimator=model, n_features_to_select=n)
    
    # Fitting the model to rfe
        X_train_rfe = rfe.fit_transform(X_train, y_train)
        X_val_rfe = rfe.transform(X_val)
    
    # Training and predicting
        model.fit(X_train_rfe, y_train)
        pred_train = model.predict(X_train_rfe)
        pred_val = model.predict(X_val_rfe)
    
    # Evaluating using the macro f1_score
        train_score = f1_score(y_train, pred_train, average="macro")
        val_score = f1_score(y_val, pred_val, average="macro")
        train_score_list.append(train_score)
        val_score_list.append(val_score)
    
    # Checking if this is the best combination of features so far
        if val_score >= high_score:
            high_score = val_score
            opt_n_features = n
            best_rfe = rfe  # Armazenar o RFE com o melhor número de features

# Checking what amount of features and which features where the best for the model
    selected_features = X_train.columns[best_rfe.support_].tolist()

    print("Optimal number of features: %d" % opt_n_features)
    print("Score with %d features: %f" % (opt_n_features, high_score))
    print("Selected Features:\n", selected_features)


In [34]:
def cv_scores(model, X, y, num_features, cat_features, num_imputing_algorithm=XGBRegressor(), cat_imputing_algorithm=XGBClassifier(), scaling_outlier= False, scaler=MinMaxScaler()):
    """
    Takes as argument the model used, the predictors and the target, the models used for imputing numerical and categorical 
    features, if any scaling and outlier removed should be performed, and what scaling method should be used.
    Then it returns the results obtained from the stratified cross-validation for the given model.
    """
    skf = StratifiedKFold(n_splits=5)

    # Generating the lists to store our results
    precision_scores_train = []
    precision_scores_val = []   
    recall_scores_train = []  
    recall_scores_val = []
    f1_scores_train = []    
    f1_scores_val = []
    index = [f'Fold {i}' for i in range(1, 6)]
    index.append("Average")
    fold = 1
    for train_index, test_index in skf.split(X, y):
        # Dividing our data in validation and train
        X_train, X_val = X.iloc[train_index].copy(), X.iloc[test_index].copy()
        y_train, y_val = y.iloc[train_index].copy(), y.iloc[test_index].copy()

        # Filling numerical missing values
        for column in num_features:
            X_train, X_val = impute_missing_values(X_train,X_val, column, num_imputing_algorithm)
        
        # Filling categorical missing values
        X_train, X_val = impute_missing_values(X_train,X_val, "Alternative Dispute Resolution", cat_imputing_algorithm)

        # Removing inconsistencies on the train
        inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
        X_train = X_train.loc[~X_train.index.isin(inconsistent)]
        y_train = y_train.loc[~y_train.index.isin(inconsistent)]

        # Performing scaling and outlier treatment dependent on the boolean
        if scaling_outlier:
            for column in num_features:
                handle_outliers(X_train, column)
                X_train, X_val = impute_missing_values(X_train,X_val, column, num_imputing_algorithm,validation= False)

            for column in num_features:
                scale_numerical(column, X_train, X_val, scaler)
        # Creating an ordinal variable
        claim_carrier_categories(X_train, X_val)

        X_val, X_train = impute_missing_values(X_val,X_train, "Carrier Claim Category", cat_imputing_algorithm, validation=False)
        check_missing_values(X_train, "Antes do modelo (train)")
        check_missing_values(X_val, "Antes do modelo (validation)")

        # Categorical Prop Encoding
        for cat_feature in cat_features:
            categorical_prop_encode(X_train, X_val, cat_feature)
            if scaling_outlier:
                scale_numerical("Carrier Claim Category", X_train, X_val, scaler)

        # Training the classification model
        model.fit(X_train, y_train)

        # Making the predictions for the training and validation data
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)

        # Calculating and storing the scores
        precision_scores_train.append(precision_score(y_train, pred_train, average='macro'))
        precision_scores_val.append(precision_score(y_val, pred_val, average='macro'))
        recall_scores_train.append(recall_score(y_train, pred_train, average='macro'))
        recall_scores_val.append(recall_score(y_val, pred_val, average='macro'))
        f1_scores_train.append(f1_score(y_train, pred_train, average='macro'))
        f1_scores_val.append(f1_score(y_val, pred_val, average='macro'))

        #feedback of which fold we are in
        print(fold)
        fold+=1
        
    precision_scores_train.append(mean(precision_scores_train))
    precision_scores_val.append(mean(precision_scores_val))
    recall_scores_train.append(mean(recall_scores_train))
    recall_scores_val.append(mean(recall_scores_val))
    f1_scores_train.append(mean(f1_scores_train))
    f1_scores_val.append(mean(f1_scores_val))

    # Storing the results in a dataframe
    model_results = pd.DataFrame(data={
        'Train_precision': precision_scores_train,
        'Test_precision': precision_scores_val,
        'Train_recall': recall_scores_train,
        'Test_recall': recall_scores_val,
        'Train_f1_score': f1_scores_train,
        'Test_f1_score': f1_scores_val,
    }, index=index)

    return model_results

In [35]:
def test_prediction(model, X, y, num_features, cat_features, data_test, 
                    num_imputing_algorithm=XGBRegressor(), 
                    cat_imputing_algorithm=XGBClassifier(), scaling_outlier = False , 
                    scaler=MinMaxScaler()):

    # Etapa 1: Imputação inicial de valores ausentes
    for column in num_features:
        X,data_test=impute_missing_values(X,data_test,column, num_imputing_algorithm)
    X,data_test=impute_missing_values(X,data_test, "Alternative Dispute Resolution", cat_imputing_algorithm)

    check_missing_values(X, "Apos tratar de missing values (train)")
    check_missing_values(data_test, "Apos tratar de missing values (validation)")

    # Etapa 2: Remoção de inconsistências
    inconsistent = X[(X['Age at Injury'] > 80) | (X["Age at Injury"] < 16)].index
    X.drop(inconsistent, inplace=True)
    y.drop(inconsistent, inplace=True)

    if scaling_outlier:
        # Etapa 3: Tratamento de outliers
        for column in num_features:
            handle_outliers(X, column)
            check_missing_values(X, "Apos handle_outliers (train)")
            X,data_test=impute_missing_values(X,data_test, column, num_imputing_algorithm,validation = False)

        for column in num_features:
            scale_numerical(column, X, data_test, scaler)
        
        check_missing_values(X, "Apos scalling (train)")
        check_missing_values(data_test, "Apos scalling (validation)")
    
    # Creating an ordinal variable
    claim_carrier_categories(X, data_test)
    data_test,X=impute_missing_values(data_test,X, "Carrier Claim Category", cat_imputing_algorithm,validation= False)

    # Categorical Prop Encoding
    for cat_feature in cat_features:
        categorical_prop_encode(X, data_test, cat_feature)
        if scaling_outlier:
            scale_numerical(cat_feature,X,data_test,scaler)
    
    check_missing_values(X, "Antes do modelo (train)")
    check_missing_values(data_test, "Antes do modelo (validation)")

    model.fit(X, y)
    pred_test = model.predict(data_test)
    pred_test = le.inverse_transform(pred_test)

    submission_df = pd.DataFrame({
        "Claim Injury Type": pred_test
    }, index=data_test.index)
    
    return submission_df

Temos de rever as num features e cat features

In [36]:
num_features = ['Age at Injury', 'Average Weekly Wage', 'Birth Year', 'IME-4 Count', 'Number of Dependents',
                "Accident Year","Accident Month","Accident Day","Accident DayOfWeek","Assembly Date DSA",
                "C-2 Date DSA","C-3 Date DSA","First Hearing Date DSA" ,"Accident Year",
    "Accident Month",
    "Accident Day",
    "Accident DayOfWeek","Accident Date","C-3 Date","First Hearing Date"]

In [37]:
cat_features = [
    "Alternative Dispute Resolution",
    "Carrier Name",
    "Carrier Type",
    "County of Injury",
    "District Name",
    "Gender",
    "Industry Code",
    "Medical Fee Region",
    "WCIO Cause of Injury Code",
    "WCIO Nature of Injury Code",
    "WCIO Part Of Body Code",
    "Age at Injury Category",
    "Carrier Claim Category",
    "Body Section"
]

temos de dropar as colunas e por no pre-processing

In [38]:
# Dropping redundant variables that carry almost the same information (are extremely correlated (|0.8|))
# We believe it was better to keep Age at Injury than birth year since it should be more related to the injury claim type (it will be tested later)
# The same logic was applied to dropping the other two dates and two DSA variables since we believe Accident date to be more important
# Para `data`
data = data.loc[:, ~data.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]

# Para `data_test`
data_test = data_test.loc[:, ~data_test.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]


for col in ['Birth Year',"Assembly Date DSA", "First Hearing Date DSA"]:
    num_features.remove(col)

In [39]:
#Since the codes always seem to provide the same or more information than the descriptions (have more categories),
#and the codes are consistent (always only having 1 description for code, while descriptions may have multiple codes)
#we will drop the description columns.
data.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)
data_test.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)


In [40]:
# Removing Zip Code for reason meantion in pre-processement
data.drop(['Zip Code'], axis=1, inplace = True)
data_test.drop(['Zip Code'], axis=1 , inplace = True)

In [41]:
X = data.drop(["Claim Injury Type"], axis = 1)

In [42]:
y = data["Claim Injury Type"]

In [43]:
# ------------------------

In [44]:
xgb_results = cv_scores(XGBClassifier(), X, y,num_features,cat_features,scaling_outlier = False)
# look at results


Antes do modelo (train): Valores ausentes restantes:
Series([], dtype: int64)

Antes do modelo (validation): Valores ausentes restantes:
Series([], dtype: int64)
1


XGBoostError: value 0 for Parameter num_class should be greater equal to 1
num_class: Number of output class in the multi-class classification.

In [ ]:
#run your model
"your model"_results

,Train_precision,Test_precision,Train_recall,Test_recall,Train_f1_score,Test_f1_score
Fold 1,0.566294,0.260142,0.368327,0.255722,0.404653,0.243337
Fold 2,0.592111,0.316560,0.363589,0.261721,0.395992,0.268905
Fold 3,0.727042,0.261532,0.367406,0.246625,0.401772,0.250295
Fold 4,0.590960,0.257523,0.368935,0.244976,0.403256,0.246095
Fold 5,0.568036,0.343732,0.363698,0.247904,0.395528,0.261758
Average,0.608888,0.287898,0.366391,0.251390,0.400240,0.254078


In [ ]:
# Test prediction
    # Choose best model from KFold above
    # Train model on whole train and predict test data
submission = test_prediction(XGBClassifier(),X,y,num_features,cat_features,data_test)
submission


Apos tratar de missing values (train): Valores ausentes restantes:
Series([], dtype: int64)

Apos tratar de missing values (validation): Valores ausentes restantes:
Series([], dtype: int64)

Antes do modelo (train): Valores ausentes restantes:
Series([], dtype: int64)

Antes do modelo (validation): Valores ausentes restantes:
Series([], dtype: int64)


KeyboardInterrupt: 

In [ ]:
submission.to_csv("Submission.csv")

In [ ]:
submission.nunique()

Claim Injury Type    7
dtype: int64